# Лаб 2. Матричные разложения

## Теория

Суть методов, основанных на матричном разложении состоит в том, что каждому пользователю $u$ ставится в соответствие вектор интересов $p_u$ длины $k$, а каждому айтему $i$ ставится в соответствие вектор интересов $q_i$, которым он удовлетворяет. Таким образом, что скалярное произведение этих векторов даёт предсказание $\hat r_{ui}$:

$$
\hat r_{ui} = p_u \dot q_i
$$

Такие вектора $p_u$ и $q_i$ составляют матрицы $P$ и $Q$ соответственно, которые в произведении даёт матрицу предсказаний $\hat R$.

![Разложение матриц](./images/PQ.drawio.png)

## Код

Ставим библиотеки

In [1]:
#%pip install --quiet -U scikit-surprise

Импорты

In [2]:
from tqdm.notebook import tqdm
import pandas as pd

# Реализации методов матричного разложения будем использовать из библиотеки surprise
from surprise import Dataset, SVD, NMF, Reader
# Кроссвалидацию проходили на машинном обучении
from surprise.model_selection import cross_validate

Загрузка датасета (из первой лабораторной)

In [3]:
data = Dataset.load_from_file('./ml-latest-small/ratings.csv', Reader(sep=',', line_format="user item rating timestamp", skip_lines=1))
trainset = data.build_full_trainset()

user_id = '42'
item_id = '50'

### SVD

In [4]:
# Обучаем модель
algo_svd = SVD(biased=False, n_factors=100)
algo_svd = algo_svd.fit(trainset)

In [5]:
# Даём предсказание для юзера и произвольного айтема
algo_svd.predict(user_id, item_id).est

4.738756425259755

Матричное представление пользователей

In [6]:
algo_svd.pu.shape

(610, 100)

In [7]:
#algo_svd.pu

Матричное представление товаров

In [8]:
algo_svd.qi.shape

(9724, 100)

In [9]:
#algo_svd.qi

Пример связи матричных представлений и итогово предсказания

In [10]:
svd_item_id = algo_svd.trainset.to_inner_iid(item_id)
svd_user_id = algo_svd.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_svd.pu[svd_user_id]*algo_svd.qi[svd_item_id]).sum()

4.738756425259755

In [11]:
algo_svd.pu[svd_user_id].shape,algo_svd.qi[svd_item_id].shape,svd_item_id,svd_user_id

((100,), (100,), 4, 41)

### Non-negative Matrix Factorization

По интерфейсу и внутреннему устройству этот метод похож на SVD. Повторите предыдущие шаги для него.

In [12]:
algo_nmf = NMF(n_factors=15)
algo_nmf = algo_nmf.fit(trainset)

In [13]:
algo_nmf.predict(user_id, item_id).est

4.415228684249061

In [14]:
nmf_item_id = algo_nmf.trainset.to_inner_iid(item_id)
nmf_user_id = algo_nmf.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_nmf.pu[nmf_user_id]*algo_nmf.qi[nmf_item_id]).sum()

4.415228684249061

### Валидация и обучение
Используя кроссвалидацию, сравните представленные алгоритмы по качеству.

Подберите гиперпараметры моделей для улучшения результатов.

Расшифровка метрик ошибок и альтернативы можно найти в документации (как и параметры моделей): https://surprise.readthedocs.io/en/stable/accuracy.html#module-surprise.accuracy

In [15]:
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9682  0.9864  0.9765  0.9804  0.9724  0.9768  0.0063  
MAE (testset)     0.7472  0.7589  0.7530  0.7600  0.7484  0.7535  0.0052  
Fit time          3.58    3.74    3.73    2.95    3.38    3.48    0.30    
Test time         0.66    0.47    0.42    0.42    0.26    0.45    0.13    


{'test_rmse': array([0.96816935, 0.98637135, 0.97649756, 0.98041693, 0.97239542]),
 'test_mae': array([0.74724031, 0.75892677, 0.7530403 , 0.75999585, 0.74841812]),
 'fit_time': (3.5835390090942383,
  3.740236759185791,
  3.732506513595581,
  2.946532726287842,
  3.379998207092285),
 'test_time': (0.6585943698883057,
  0.4706535339355469,
  0.4167215824127197,
  0.4237692356109619,
  0.2562887668609619)}

In [16]:
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9178  0.9180  0.9191  0.9203  0.9232  0.9196  0.0020  
MAE (testset)     0.7031  0.7036  0.7029  0.7026  0.7093  0.7043  0.0025  
Fit time          4.92    5.94    7.37    7.11    6.97    6.46    0.91    
Test time         0.21    0.66    0.31    0.47    0.66    0.46    0.18    


{'test_rmse': array([0.91777801, 0.91795881, 0.91905987, 0.92025754, 0.92316236]),
 'test_mae': array([0.70309853, 0.7035936 , 0.702925  , 0.70255308, 0.70925284]),
 'fit_time': (4.9231860637664795,
  5.935331106185913,
  7.365713834762573,
  7.108931303024292,
  6.971379041671753),
 'test_time': (0.2114424705505371,
  0.6552667617797852,
  0.31474995613098145,
  0.46799492835998535,
  0.6590111255645752)}

In [17]:
from tqdm.notebook import tqdm
svd_rmse={}
start=0
stop=200
step=1
'''
with tqdm(total=(stop-start)/step) as pbar:
    for i in range(start,stop,step):
        algo_svd = SVD(biased=False, n_factors=i)
        algo_svd = algo_svd.fit(trainset)
        svd_rmse[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()
        pbar.update(1)
dict(sorted(svd_rmse.items(), key=lambda item: item[1], reverse=False))
'''

"\nwith tqdm(total=(stop-start)/step) as pbar:\n    for i in range(start,stop,step):\n        algo_svd = SVD(biased=False, n_factors=i)\n        algo_svd = algo_svd.fit(trainset)\n        svd_rmse[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()\n        pbar.update(1)\ndict(sorted(svd_rmse.items(), key=lambda item: item[1], reverse=False))\n"

In [18]:
nmf_rmse={}
start=0
stop=200
step=1
'''
with tqdm(total=(stop-start)/step) as pbar:
    for i in range(start,stop,step):
        algo_nmf = SVD(biased=False, n_factors=i)
        algo_nmf = algo_svd.fit(trainset)
        nmf_rmse[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()
        pbar.update(1)
dict(sorted(nmf_rmse.items(), key=lambda item: item[1], reverse=False))
'''

"\nwith tqdm(total=(stop-start)/step) as pbar:\n    for i in range(start,stop,step):\n        algo_nmf = SVD(biased=False, n_factors=i)\n        algo_nmf = algo_svd.fit(trainset)\n        nmf_rmse[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()\n        pbar.update(1)\ndict(sorted(nmf_rmse.items(), key=lambda item: item[1], reverse=False))\n"

In [19]:
algo_svd = SVD(biased=False, n_factors=8)
algo_svd = algo_svd.fit(trainset)
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9346  0.9360  0.9393  0.9363  0.9291  0.9351  0.0033  
MAE (testset)     0.7134  0.7137  0.7163  0.7195  0.7129  0.7152  0.0025  
Fit time          1.27    1.52    1.55    1.54    1.44    1.46    0.10    
Test time         0.34    0.40    0.65    0.54    0.43    0.47    0.11    


{'test_rmse': array([0.9345658 , 0.93597251, 0.93934917, 0.93630161, 0.92914017]),
 'test_mae': array([0.71338001, 0.71374133, 0.71633934, 0.71950133, 0.7129029 ]),
 'fit_time': (1.2732622623443604,
  1.5172233581542969,
  1.551271677017212,
  1.5444691181182861,
  1.4356036186218262),
 'test_time': (0.3426930904388428,
  0.3991565704345703,
  0.6477072238922119,
  0.5393025875091553,
  0.4322068691253662)}

In [20]:
algo_nmf = NMF(biased=False,n_factors=23)
algo_nmf = algo_nmf.fit(trainset)
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9259  0.9262  0.9258  0.9198  0.9262  0.9248  0.0025  
MAE (testset)     0.7022  0.7012  0.7019  0.6970  0.7014  0.7007  0.0019  
Fit time          8.72    8.46    8.48    8.31    8.85    8.56    0.19    
Test time         0.71    0.46    0.74    0.67    0.44    0.60    0.13    


{'test_rmse': array([0.925866  , 0.92621683, 0.92578756, 0.91975933, 0.92624007]),
 'test_mae': array([0.70220515, 0.7012175 , 0.70191663, 0.6969686 , 0.7013571 ]),
 'fit_time': (8.720702886581421,
  8.462809324264526,
  8.477094173431396,
  8.314127445220947,
  8.849178791046143),
 'test_time': (0.7141070365905762,
  0.4574470520019531,
  0.7419037818908691,
  0.6660783290863037,
  0.4379432201385498)}

## Призрак в машине

Проинтерпретируйте получившиеся "интересы" в модели NnMF.

Например, можно начать с просмотра отсортированных по ним фильмов

In [21]:
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')
# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

In [22]:
factor_idx = 0
items = list(algo_nmf.trainset.all_items())

items.sort(key=lambda inner_id: -algo_nmf.qi[inner_id][factor_idx])
named_items = [[id_to_title.loc[int(trainset.to_raw_iid(k))]["title"], [int(f*100) for f in algo_nmf.qi[k]]] for k in items]
#named_items

In [23]:
for i in range (5):
    print(named_items[i][0],'\t\t',movies.loc[(movies['title'] == named_items[i][0])]['genres'].reset_index(drop=True)[0])
print("-----------")
for i in range (5,0,-1):
    print(named_items[-i][0],'\t\t',movies.loc[(movies['title'] == named_items[-i][0])]['genres'].reset_index(drop=True)[0])  

High Sierra (1941) 		 Crime|Drama|Film-Noir|Thriller
Any Which Way You Can (1980) 		 Comedy
Girl Who Leapt Through Time, The (Toki o kakeru shôjo) (2006) 		 Animation|Comedy|Drama|Romance|Sci-Fi
Band of Brothers (2001) 		 Action|Drama|War
Kung Fu Panda 2 (2011) 		 Action|Adventure|Animation|Children|Comedy|IMAX
-----------
Rope (1948) 		 Crime|Drama|Thriller
Christiane F. (a.k.a. We Children from Bahnhof Zoo) (Christiane F. - Wir Kinder vom Bahnhof Zoo) (1981) 		 Drama
Monster (2003) 		 Crime|Drama
My First Mister (2001) 		 Comedy|Drama
Skin I Live In, The (La piel que habito) (2011) 		 Drama


## вывод
разделение по первому признаку разделяет выборку явно, с положительным знаком преимущественно комедии, детские и семейные фильмы, а с отрицательным знаком преимущественно драмы.

## Адаптация собственного алгоритма

Адаптируйте ваш алгоритм из первой лабораторной для совместимости с библиотекой согласно документации: https://surprise.readthedocs.io/en/stable/building_custom_algo.html

Сравните настроенные ранее модели со своим решением с помощью кроссвалидации.

In [42]:
from surprise import AlgoBase, Dataset
from surprise.model_selection import cross_validate
from surprise import PredictionImpossible
f=int(0)
class MyOwnAlgorithm(AlgoBase):
    
    def __init__(self):
        AlgoBase.__init__(self)
        self.f=0
        self.count=0
        self.s={}
        self.u_mid ={}
        self.ui_filmid={}
        self.ui_value={}   
        self.norm_dict={}
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        #if self.f==0:
        for key, value in trainset.ur.items() :
            self.ui_filmid[key]=[]
            self.ui_value[key]=[]
            norm_dict_small={}
            for i in range (len(trainset.ur[key])):
                self.ui_filmid[key].append( value[i][0])
                self.ui_value[key].append(value[i][1])
                norm_dict_small[value[i][0]]=value[i][1]
            self.norm_dict[key]=norm_dict_small
        #print(0 in self.norm_dict[0])
            
        for key, value in trainset.ur.items() :
            s_small={}
            
            sum_u=0
            s_downk=0
            s_downl=0
            for k in range (len(trainset.ur[key])):
                sum_u+=trainset.ur[key][k][1]
            s_downk= sum(i*i for i in self.ui_value[key])
            for l, e in trainset.ur.items():
                if (l!=key):
                    s_up=0
                    #print(key, l)
                    s_downl=sum(i*i for i in self.ui_value[l])
                    a= list(set( self.ui_filmid[key])& set(self.ui_filmid[l]))
                    for j in a:
                        s_up+=self.ui_value[key][self.ui_filmid[key].index(j)]*self.ui_value[l][self.ui_filmid[l].index(j)]
                    s_small[l]=s_up/((s_downk*s_downl)**0.5) 
            self.s[key]=s_small
            self.u_mid[key]=sum_u/len(trainset.ur[key])
        self.f+=1     
        print('-------')
        return self
    
    def estimate(self, u, i):
        self.count+=1
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible("User and/or item is unknown.")
        r_up=0
        r_down=0
        for l, e in trainset.ur.items():
            if (l!=u and i in self.norm_dict[l]):
                r_up+=self.s[u][l]*(self.norm_dict[l][i]-self.u_mid[l])
                r_down+=abs(self.s[u][l])
        if r_down==0:
            bsl=0.0
        else:
            bsl = self.u_mid[l]+r_up/r_down
        if self.count%10000==0:
            print (self.f,self.count)
        return bsl

In [43]:
#data = Dataset.load_builtin("ml-100k")
algo = MyOwnAlgorithm()


cross_validate(algo, data, verbose=True)

-------
1 10000
1 20000
-------
1 30000
1 40000
-------
1 50000
1 60000
-------
1 70000
1 80000
-------
1 90000
1 100000
Evaluating RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1229  1.0205  1.1735  1.0705  1.0279  1.0831  0.0581  
MAE (testset)     0.9291  0.7774  0.8969  0.8131  0.7787  0.8390  0.0626  
Fit time          124.71  80.47   86.88   130.06  133.00  111.02  22.58   
Test time         12.83   6.78    18.13   13.66   12.46   12.77   3.62    


{'test_rmse': array([1.12286482, 1.02045944, 1.17349142, 1.07051806, 1.02793518]),
 'test_mae': array([0.92913867, 0.77737825, 0.8969373 , 0.81308768, 0.77866609]),
 'fit_time': (124.70506858825684,
  80.47050523757935,
  86.87774658203125,
  130.0576868057251,
  132.99581360816956),
 'test_time': (12.832032203674316,
  6.779748201370239,
  18.132147550582886,
  13.663106918334961,
  12.457188606262207)}

Evaluating RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1229  1.0205  1.1735  1.0705  1.0279  1.0831  0.0581  
MAE (testset)     0.9291  0.7774  0.8969  0.8131  0.7787  0.8390  0.0626  
Fit time          124.71  80.47   86.88   130.06  133.00  111.02  22.58   
Test time         12.83   6.78    18.13   13.66   12.46   12.77   3.62    
{'test_rmse': array([1.12286482, 1.02045944, 1.17349142, 1.07051806, 1.02793518]),
 'test_mae': array([0.92913867, 0.77737825, 0.8969373 , 0.81308768, 0.77866609]),
 'fit_time': (124.70506858825684,
  80.47050523757935,
  86.87774658203125,
  130.0576868057251,
  132.99581360816956),
 'test_time': (12.832032203674316,
  6.779748201370239,
  18.132147550582886,
  13.663106918334961,
  12.457188606262207)}

In [44]:
cross_validate(algo_svd, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9332  0.9371  0.9241  0.9416  0.9356  0.9343  0.0058  
MAE (testset)     0.7142  0.7176  0.7065  0.7188  0.7158  0.7146  0.0043  
Fit time          2.66    1.60    1.22    1.29    1.20    1.59    0.55    
Test time         0.50    0.33    0.77    0.33    0.31    0.45    0.17    


{'test_rmse': array([0.9331839 , 0.93710614, 0.9241298 , 0.94159087, 0.93562942]),
 'test_mae': array([0.71423389, 0.71761982, 0.70653767, 0.71882104, 0.71581922]),
 'fit_time': (2.6617445945739746,
  1.6021904945373535,
  1.21748685836792,
  1.2884337902069092,
  1.2016332149505615),
 'test_time': (0.4981529712677002,
  0.32650017738342285,
  0.7655737400054932,
  0.326524019241333,
  0.3144805431365967)}

In [45]:
cross_validate(algo_nmf, data, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9202  0.9151  0.9195  0.9360  0.9201  0.9222  0.0072  
MAE (testset)     0.6989  0.6916  0.6966  0.7096  0.6982  0.6990  0.0059  
Fit time          7.76    12.13   12.74   12.58   9.22    10.88   2.02    
Test time         0.59    0.65    1.26    0.59    0.34    0.69    0.30    


{'test_rmse': array([0.9202265 , 0.91506701, 0.91949515, 0.93599235, 0.92007315]),
 'test_mae': array([0.69887322, 0.69162585, 0.69660227, 0.70958077, 0.69816911]),
 'fit_time': (7.75804591178894,
  12.12675428390503,
  12.739007949829102,
  12.576953887939453,
  9.22347617149353),
 'test_time': (0.5904397964477539,
  0.6492345333099365,
  1.256413221359253,
  0.5905928611755371,
  0.34058260917663574)}

In [46]:
algo.predict(user_id, item_id).est

4.555112571354297

In [47]:
algo_nmf.predict(user_id, item_id).est

4.74747574124056

In [48]:
algo_svd.predict(user_id, item_id).est

4.875679112291458

# Вывод
ошибка полученной модели примерно такая же, как при стандартных значениях начальной модели (n_factors default), но итоговый результат не так хорош, как хотелось бы